# ▶️ YouTube Video Analysis
### Author: Aston Crawley

In [2]:
# ============== 1. Setup ==============
# Import libraries
import os
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import nltk

In [4]:
# ============== 2. Data Loading ==============
# Load YouTube US comments data skipping corrupted rows
df = pd.read_csv(os.path.join("..", "data", "UScomments.csv"), on_bad_lines = "skip")

/var/folders/kh/hv0681hd5n1bnsmh2y16qm3h0000gn/T/ipykernel_20491/3347441801.py:3: DtypeWarning: Columns (2,3) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(os.path.join("..", "data", "UScomments.csv"), on_bad_lines = "skip")


In [11]:
# Preview data
print(df.head())
print(df.info())
print(df.describe())

      video_id                                       comment_text likes  \
0  XpVt6Z1Gjjo                  Logan Paul it's yo big day ‼️‼️‼️     4   
1  XpVt6Z1Gjjo  I've been following you from the start of your...     3   
2  XpVt6Z1Gjjo                 Say hi to Kong and maverick for me     3   
3  XpVt6Z1Gjjo                                MY FAN . attendance     3   
4  XpVt6Z1Gjjo                                         trending 😉     3   

  replies  
0       0  
1       0  
2       0  
3       0  
4       0  
<class 'pandas.core.frame.DataFrame'>
Index: 691374 entries, 0 to 691399
Data columns (total 4 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   video_id      691374 non-null  object
 1   comment_text  691374 non-null  object
 2   likes         691374 non-null  object
 3   replies       691374 non-null  object
dtypes: object(4)
memory usage: 26.4+ MB
None
           video_id comment_text   likes  replies
count        69137

In [12]:
# ============== 3. Data Cleaning ==============
# Check for missing values
print(df.isnull().values.any())

# Check location and quantity of missing values
print(df.isnull().sum())

# Handle missing values
df.dropna(inplace = True)

False
video_id        0
comment_text    0
likes           0
replies         0
dtype: int64


In [ ]:
# ============== 4. Exploratory Data Analysis (EDA) ==============
# Sentiment analysis
